In [3]:
import pandas as pd
import numpy as np

# -------------------- Parameters --------------------
CL = 0.99
# z = 2.3263478740408408  # (not used directly in MC; kept for reference)
total_value = 1_000_000.0
weights = np.array([0.5, 0.3, 0.2])

N_SIMS = 40_000        # number of Monte Carlo scenarios
N_STEPS_10D = 10       # steps for 10-day horizon
RNG = np.random.default_rng(12345)  # set seed for reproducibility

# -------------------- Load returns --------------------
# If running locally, replace the path with your local path to the CSV
returns = pd.read_csv("sample_data/3assets_returns.csv", index_col=0, parse_dates=True)
assets = list(returns.columns)
assert len(assets) == 3, f"Expected 3 assets, got {len(assets)}: {assets}"

# Estimate daily mean & covariance from historical data
mu = returns.mean().values           # shape (3,)
Sigma = returns.cov().values         # shape (3,3)

# -------------------- Helper: VaR & ES from loss array --------------------
alpha = CL

def var_es(losses: np.ndarray, alpha: float = alpha) -> tuple[float, float]:
    """
    Compute VaR and ES at level alpha from a 1D array of losses.
    losses: positive = loss, higher = worse.
    """
    var = float(np.percentile(losses, alpha * 100))
    tail = losses[losses >= var]
    es = float(tail.mean()) if tail.size > 0 else float(var)
    return var, es

# -------------------- 1-day Monte Carlo --------------------
# Draw MVN returns for one day: shape (N_SIMS, 3)
ret_1d = RNG.multivariate_normal(mean=mu, cov=Sigma, size=N_SIMS)

# Losses (currency) for assets and portfolio
losses_1d_assets = -ret_1d * total_value                      # (N,3)
port_ret_1d = ret_1d @ weights                                # (N,)
losses_1d_port = -port_ret_1d * total_value                   # (N,)

# Compute VaR/ES per asset and portfolio (1d)
var1d_assets, es1d_assets = [], []
for j in range(3):
    v, e = var_es(losses_1d_assets[:, j])
    var1d_assets.append(v)
    es1d_assets.append(e)
var1d_port, es1d_port = var_es(losses_1d_port)

# -------------------- 10-day Monte Carlo --------------------
# Simulate 10-day paths explicitly and sum daily returns per scenario
# paths shape: (N_SIMS, N_STEPS_10D, 3)
paths = RNG.multivariate_normal(mean=mu, cov=Sigma, size=(N_SIMS, N_STEPS_10D))
ret_10d = paths.sum(axis=1)                                   # (N,3)

losses_10d_assets = -ret_10d * total_value                    # (N,3)
port_ret_10d = ret_10d @ weights                              # (N,)
losses_10d_port = -port_ret_10d * total_value                 # (N,)

var10d_assets, es10d_assets = [], []
for j in range(3):
    v, e = var_es(losses_10d_assets[:, j])
    var10d_assets.append(v)
    es10d_assets.append(e)
var10d_port, es10d_port = var_es(losses_10d_port)

# -------------------- Results table --------------------
results = pd.DataFrame({
    "VaR_1d":  var1d_assets + [var1d_port],
    "ES_1d":   es1d_assets + [es1d_port],
    "VaR_10d": var10d_assets + [var10d_port],
    "ES_10d":  es10d_assets + [es10d_port],
}, index=assets + ["Portfolio"])

# Show and save
print(results.round(2))
results.round(2).to_csv("sample_data/var_es_montecarlo_40k_results.csv")


                  VaR_1d     ES_1d    VaR_10d     ES_10d
Asset_A_Return  26035.85  29885.65   68939.79   81842.76
Asset_B_Return  16751.79  19234.20   50820.46   58863.00
Asset_C_Return  43516.01  49859.22  123005.28  144478.70
Portfolio       18886.34  21622.58   49539.72   59108.34
